In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/ml-final/ML-Final')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from prophet import Prophet
import pmdarima as pm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import mlflow
import mlflow.sklearn
import dagshub
import joblib
import pickle
import os
from statsmodels.tsa.stattools import adfuller, kpss

warnings.filterwarnings('ignore')

In [4]:
dagshub.init(repo_owner='egval20', repo_name='ML-Final', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=48525ef0-5d1e-4e66-9586-7989e708b032&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=01c13604b8b70cf35050bb3f23ebc40efa279baeb343f3b0d3181d7c0078dce4




Accessing as egval20

Initialized MLflow to track repo "egval20/ML-Final"

Repository egval20/ML-Final initialized!

## Data Prep

In [5]:
import joblib
import sys
sys.path.append('.')
from data_preprocessing_pipeline import *

In [6]:
def get_model_ready_data(pipeline_path='preprocessing_pipeline.pkl'):
    pipeline = joblib.load(pipeline_path)
    def preprocess_for_model(raw_data):
        return pipeline.transform(raw_data)
    return preprocess_for_model, pipeline

preprocess_fn, loaded_pipeline = get_model_ready_data()

train_raw = pd.read_csv('data/train.csv')
test_raw = pd.read_csv('data/test.csv')
stores = pd.read_csv('data/stores.csv')
features = pd.read_csv('data/features.csv')
print(f"Train shape: {train_raw.shape}")
print(f"Test shape: {test_raw.shape}")
print(f"Date range - Train: {train_raw['Date'].min()} to {train_raw['Date'].max()}")
print(f"Date range - Test: {test_raw['Date'].min()} to {test_raw['Date'].max()}")

train_processed = preprocess_fn(train_raw)
test_processed = preprocess_fn(test_raw)
X_train = train_processed[loaded_pipeline.feature_names_]
y_train = train_processed['Weekly_Sales']
X_test = test_processed[loaded_pipeline.feature_names_]

print(f"Preprocessed train shape: {X_train.shape}")
print(f"Preprocessed test shape: {X_test.shape}")

Train shape: (421570, 5)
Test shape: (115064, 4)
Date range - Train: 2010-02-05 to 2012-10-26
Date range - Test: 2012-11-02 to 2013-07-26
Preprocessed train shape: (421570, 62)
Preprocessed test shape: (115064, 62)


In [7]:
def drop_lag_features(data, columns_to_drop):
    existing_cols = [col for col in columns_to_drop if col in data.columns]
    cleaned_data = data.drop(columns=existing_cols)
    print(f"Dropped {len(existing_cols)} lag/MA columns: {existing_cols}")
    return cleaned_data

lag_columns_to_drop = [
    'Sales_Lag_1', 'Sales_Lag_2', 'Sales_Lag_3', 'Sales_Lag_4', 'Sales_Lag_8', 'Sales_Lag_52',
    'Sales_MA_4', 'Sales_MA_8', 'Sales_MA_12',
    'Sales_STD_4', 'Sales_STD_8', 'Sales_STD_12'
]

train_processed_clean = drop_lag_features(train_processed, lag_columns_to_drop)
test_processed_clean = drop_lag_features(test_processed, lag_columns_to_drop)

print(f"Original train shape: {train_processed.shape}")
print(f"Cleaned train shape: {train_processed_clean.shape}")

Dropped 12 lag/MA columns: ['Sales_Lag_1', 'Sales_Lag_2', 'Sales_Lag_3', 'Sales_Lag_4', 'Sales_Lag_8', 'Sales_Lag_52', 'Sales_MA_4', 'Sales_MA_8', 'Sales_MA_12', 'Sales_STD_4', 'Sales_STD_8', 'Sales_STD_12']
Dropped 12 lag/MA columns: ['Sales_Lag_1', 'Sales_Lag_2', 'Sales_Lag_3', 'Sales_Lag_4', 'Sales_Lag_8', 'Sales_Lag_52', 'Sales_MA_4', 'Sales_MA_8', 'Sales_MA_12', 'Sales_STD_4', 'Sales_STD_8', 'Sales_STD_12']
Original train shape: (421570, 64)
Cleaned train shape: (421570, 52)


In [8]:
mlflow.set_experiment("SARIMA_Training")
with mlflow.start_run(run_name="SARIMA_Cleaning"):
    mlflow.log_param("columns_to_drop_count", len(lag_columns_to_drop))
    mlflow.log_param("columns_to_drop", lag_columns_to_drop)
    mlflow.log_metric("original_train_rows", train_processed.shape[0])
    mlflow.log_metric("original_train_cols", train_processed.shape[1])
    mlflow.log_metric("final_train_rows", train_processed_clean.shape[0])
    mlflow.log_metric("final_train_cols", train_processed_clean.shape[1])

2025/07/06 14:04:36 INFO mlflow.tracking.fluent: Experiment with name 'SARIMA_Training' does not exist. Creating a new experiment.


🏃 View run SARIMA_Cleaning at: https://dagshub.com/egval20/ML-Final.mlflow/#/experiments/5/runs/b04ffc638e9742d6bcbba041c1ca1d30
🧪 View experiment at: https://dagshub.com/egval20/ML-Final.mlflow/#/experiments/5


In [14]:
class TimeSeriesAnalyzer:
    def __init__(self):
        self.stationarity_results = {}

    def check_stationarity(self, series, name="Series"):
        adf_result = adfuller(series.dropna())
        adf_pvalue = adf_result[1]
        kpss_result = kpss(series.dropna())
        kpss_pvalue = kpss_result[1]

        print(f"\n{name} Stationarity Tests:")
        print(f"ADF Test - p-value: {adf_pvalue:.4f} ({'Stationary' if adf_pvalue < 0.05 else 'Non-stationary'})")
        print(f"KPSS Test - p-value: {kpss_pvalue:.4f} ({'Stationary' if kpss_pvalue > 0.05 else 'Non-stationary'})")

        self.stationarity_results[name] = {
            'adf_pvalue': adf_pvalue,
            'kpss_pvalue': kpss_pvalue,
            'is_stationary': adf_pvalue < 0.05 and kpss_pvalue > 0.05
        }

        return adf_pvalue < 0.05 and kpss_pvalue > 0.05

    def plot_decomposition(self, series, freq=52, title="Time Series Decomposition"):
        decomposition = seasonal_decompose(series, model='additive', period=freq)
        fig, axes = plt.subplots(4, 1, figsize=(15, 12))
        decomposition.observed.plot(ax=axes[0], title=f'{title} - Original')
        decomposition.trend.plot(ax=axes[1], title='Trend')
        decomposition.seasonal.plot(ax=axes[2], title='Seasonal')
        decomposition.resid.plot(ax=axes[3], title='Residual')
        plt.tight_layout()
        plt.show()

        return decomposition

In [15]:
class WalmartTimeSeriesPreprocessor:
    def __init__(self):
        self.store_dept_scalers = {}
        self.is_fitted = False

    def prepare_time_series_data(self, data, target_col='Weekly_Sales'):
        data = data.copy()
        data['Date'] = pd.to_datetime(data['Date'])
        data = data.sort_values(['Store', 'Dept', 'Date'])
        ts_data = []
        store_dept_combinations = data.groupby(['Store', 'Dept']).size().reset_index(name='count')

        for _, row in store_dept_combinations.iterrows():
            store, dept = row['Store'], row['Dept']
            subset = data[(data['Store'] == store) & (data['Dept'] == dept)].copy()
            if len(subset) > 10:
                subset = subset.set_index('Date')
                subset = subset.sort_index()

                ts_data.append({
                    'store': store,
                    'dept': dept,
                    'data': subset,
                    'series': subset[target_col] if target_col in subset.columns else None
                })

        return ts_data

    def create_hierarchical_series(self, data, level='total'):
        data = data.copy()
        data['Date'] = pd.to_datetime(data['Date'])

        if level == 'total':
            series = data.groupby('Date')['Weekly_Sales'].sum().sort_index()
        elif level == 'store':
            series = data.groupby(['Store', 'Date'])['Weekly_Sales'].sum().reset_index()
        elif level == 'dept':
            series = data.groupby(['Dept', 'Date'])['Weekly_Sales'].sum().reset_index()
        else:
            series = data.groupby(['Store', 'Dept', 'Date'])['Weekly_Sales'].sum().reset_index()

        return series

In [16]:
ts_analyzer = TimeSeriesAnalyzer()
ts_preprocessor = WalmartTimeSeriesPreprocessor()

total_sales = ts_preprocessor.create_hierarchical_series(train_processed_clean, level='total')
print(f"Total sales series length: {len(total_sales)}")

store_sales = ts_preprocessor.create_hierarchical_series(train_processed_clean, level='store')
print(f"Store level data shape: {store_sales.shape}")

dept_sales = ts_preprocessor.create_hierarchical_series(train_processed_clean, level='dept')
print(f"Department level data shape: {dept_sales.shape}")

Total sales series length: 143
Store level data shape: (6435, 3)
Department level data shape: (11090, 3)


## Split Data and Evaluation

In [9]:
def create_time_series_splits(data, validation_size=0.2):
    data = data.copy()
    data['Date'] = pd.to_datetime(data['Date'])
    unique_dates = sorted(data['Date'].unique())
    total_periods = len(unique_dates)

    split_point = int(total_periods * (1 - validation_size))
    train_end_date = unique_dates[split_point - 1]
    val_start_date = unique_dates[split_point]

    train_data = data[data['Date'] <= train_end_date]
    val_data = data[data['Date'] >= val_start_date]

    print(f"Training period: {train_data['Date'].min()} to {train_data['Date'].max()}")
    print(f"Validation period: {val_data['Date'].min()} to {val_data['Date'].max()}")
    print(f"Training samples: {len(train_data)}, Validation samples: {len(val_data)}")

    return train_data, val_data

In [10]:
def evaluate_time_series_model(model, train_series, val_series, model_name):
    try:
        fitted_model = model.fit()

        val_steps = len(val_series)
        forecast = fitted_model.forecast(steps=val_steps)

        mae = mean_absolute_error(val_series, forecast)
        mse = mean_squared_error(val_series, forecast)
        rmse = np.sqrt(mse)

        mape = np.mean(np.abs((val_series - forecast) / val_series)) * 100

        metrics = {
            'MAE': mae,
            'MSE': mse,
            'RMSE': rmse,
            'MAPE': mape,
            'AIC': fitted_model.aic,
            'BIC': fitted_model.bic
        }

        print(f"\n{model_name} Validation Results:")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")

        return fitted_model, forecast, metrics

    except Exception as e:
        print(f"Error evaluating {model_name}: {e}")
        return None, None, None

In [18]:
train_split, val_split = create_time_series_splits(train_processed_clean, validation_size=0.2)

train_total_sales = ts_preprocessor.create_hierarchical_series(train_split, level='total')
val_total_sales = ts_preprocessor.create_hierarchical_series(val_split, level='total')

print(f"Training series length: {len(train_total_sales)}")
print(f"Validation series length: {len(val_total_sales)}")

Training period: 2010-02-05 00:00:00 to 2012-04-06 00:00:00
Validation period: 2012-04-13 00:00:00 to 2012-10-26 00:00:00
Training samples: 335761, Validation samples: 85809
Training series length: 114
Validation series length: 29


# SARIMA

In [12]:
class ARIMAModeler:
    def __init__(self):
        self.models = {}
        self.best_params = {}
        self.predictions = {}

    def auto_arima_search(self, series, seasonal=False, m=52):
        try:
            if seasonal:
                auto_model = pm.auto_arima(
                    series,
                    start_p=0, start_q=0,
                    max_p=3, max_q=3,
                    seasonal=True,
                    start_P=0, start_Q=0,
                    max_P=2, max_Q=2,
                    m=m,
                    stepwise=True,
                    suppress_warnings=True,
                    error_action='ignore'
                )
            else:
                auto_model = pm.auto_arima(
                    series,
                    start_p=0, start_q=0,
                    max_p=5, max_q=5,
                    seasonal=False,
                    stepwise=True,
                    suppress_warnings=True,
                    error_action='ignore'
                )

            return auto_model.order, auto_model.seasonal_order if seasonal else None

        except Exception as e:
            print(f"Auto ARIMA search failed: {e}")
            return (1, 1, 1), (1, 1, 1, 52) if seasonal else None

    def fit_sarima(self, series, order=None, seasonal_order=None, name="SARIMA"):
        if order is None or seasonal_order is None:
            order, seasonal_order = self.auto_arima_search(series, seasonal=True)

        print(f"Fitting SARIMA{order}x{seasonal_order} model...")

        try:
            model = SARIMAX(series, order=order, seasonal_order=seasonal_order)
            fitted_model = model.fit(disp=False)

            self.models[name] = fitted_model
            self.best_params[name] = {'order': order, 'seasonal_order': seasonal_order}
            print(f"SARIMA{order}x{seasonal_order} model fitted successfully")
            print(f"AIC: {fitted_model.aic:.2f}")
            print(f"BIC: {fitted_model.bic:.2f}")

            return fitted_model

        except Exception as e:
            print(f"SARIMA model fitting failed: {e}")
            return None

    def predict(self, model_name, steps):
        if model_name not in self.models:
            raise ValueError(f"Model {model_name} not found")
        model = self.models[model_name]
        forecast = model.forecast(steps=steps)

        self.predictions[model_name] = forecast
        return forecast

    def evaluate(self, model_name, actual_values):
        if model_name not in self.predictions:
            raise ValueError(f"No predictions found for {model_name}")

        predictions = self.predictions[model_name]
        min_len = min(len(predictions), len(actual_values))
        pred_aligned = predictions[:min_len]
        actual_aligned = actual_values[:min_len]

        mae = mean_absolute_error(actual_aligned, pred_aligned)
        mse = mean_squared_error(actual_aligned, pred_aligned)
        rmse = np.sqrt(mse)

        return {'MAE': mae, 'MSE': mse, 'RMSE': rmse}

## Training

In [19]:
with mlflow.start_run(run_name="SARIMA_Training"):
    sarima_modeler = ARIMAModeler()

    mlflow.log_param("model_type", "SARIMA")
    mlflow.log_param("seasonal_period", 52)
    mlflow.log_param("validation_size", 0.2)

    print("\n1. Training SARIMA on Total Sales with Validation")
    order, seasonal_order = sarima_modeler.auto_arima_search(train_total_sales, seasonal=True, m=52)
    print(f"Optimal SARIMA order: {order}x{seasonal_order}")
    sarima_model = SARIMAX(train_total_sales, order=order, seasonal_order=seasonal_order)
    fitted_sarima, sarima_forecast, sarima_metrics = evaluate_time_series_model(
        sarima_model, train_total_sales, val_total_sales, "SARIMA_Total"
    )

    if fitted_sarima:
        mlflow.log_param("sarima_order", str(order))
        mlflow.log_param("sarima_seasonal_order", str(seasonal_order))
        for metric_name, metric_value in sarima_metrics.items():
            mlflow.log_metric(f"total_{metric_name.lower()}", metric_value)
        final_sarima = SARIMAX(
            pd.concat([train_total_sales, val_total_sales]),
            order=order,
            seasonal_order=seasonal_order
        ).fit(disp=False)
        sarima_modeler.models["SARIMA_Total"] = final_sarima
        joblib.dump(final_sarima, 'models/sarima_total_model.pkl')
        mlflow.log_artifact('models/sarima_total_model.pkl')

    print("\n2. Training SARIMA for Top 5 Departments with Validation")
    top_depts = train_split.groupby('Dept')['Weekly_Sales'].sum().nlargest(5).index

    dept_sarima_models = {}
    dept_metrics = {}

    for dept in top_depts:
        print(f"\nTraining SARIMA for Department {dept}")
        dept_train_data = train_split[train_split['Dept'] == dept]
        dept_val_data = val_split[val_split['Dept'] == dept]
        if len(dept_train_data) > 104 and len(dept_val_data) > 10:
            dept_train_series = dept_train_data.groupby('Date')['Weekly_Sales'].sum().sort_index()
            dept_val_series = dept_val_data.groupby('Date')['Weekly_Sales'].sum().sort_index()
            order, seasonal_order = sarima_modeler.auto_arima_search(dept_train_series, seasonal=True, m=52)
            dept_model = SARIMAX(dept_train_series, order=order, seasonal_order=seasonal_order)
            fitted_dept_model, dept_forecast, dept_metrics_dict = evaluate_time_series_model(
                dept_model, dept_train_series, dept_val_series, f"SARIMA_Dept_{dept}"
            )

            if fitted_dept_model:
                dept_metrics[dept] = dept_metrics_dict
                for metric_name, metric_value in dept_metrics_dict.items():
                    mlflow.log_metric(f"dept_{dept}_{metric_name.lower()}", metric_value)
                combined_dept_data = pd.concat([
                    train_split[train_split['Dept'] == dept],
                    val_split[val_split['Dept'] == dept]
                ])
                combined_dept_series = combined_dept_data.groupby('Date')['Weekly_Sales'].sum().sort_index()
                final_dept_model = SARIMAX(
                    combined_dept_series,
                    order=order,
                    seasonal_order=seasonal_order
                ).fit(disp=False)
                dept_sarima_models[dept] = final_dept_model
        else:
            print(f"Insufficient data for department {dept}")

    joblib.dump(dept_sarima_models, 'models/sarima_dept_models.pkl')
    mlflow.log_artifact('models/sarima_dept_models.pkl')

    print("\nSARIMA Training with Validation Complete!")


1. Training SARIMA on Total Sales with Validation
Optimal SARIMA order: (1, 0, 0)x(0, 0, 2, 52)

SARIMA_Total Validation Results:
MAE: 3571804.8588
MSE: 18344869560249.2578
RMSE: 4283091.1221
MAPE: 7.7180
AIC: 3887.1330
BIC: 3898.0778

2. Training SARIMA for Top 5 Departments with Validation

Training SARIMA for Department 92

SARIMA_Dept_92 Validation Results:
MAE: 243941.7449
MSE: 73919409402.6867
RMSE: 271881.2414
MAPE: 7.0448
AIC: 1634.6303
BIC: 1640.9630

Training SARIMA for Department 95

SARIMA_Dept_95 Validation Results:
MAE: 95604.7588
MSE: 14928918130.9655
RMSE: 122183.9520
MAPE: 2.8859
AIC: 3067.5963
BIC: 3081.2773

Training SARIMA for Department 38

SARIMA_Dept_38 Validation Results:
MAE: 279709.2610
MSE: 88382999515.2511
RMSE: 297292.7842
MAPE: 10.3898
AIC: 3069.6343
BIC: 3080.5438

Training SARIMA for Department 72

SARIMA_Dept_72 Validation Results:
MAE: 778724.1958
MSE: 680837982029.4703
RMSE: 825129.0699
MAPE: 43.9891
AIC: 3538.2011
BIC: 3549.1459

Training SARIMA for